In [6]:
!pip install -q lmdb
!pip install -q fire

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [1]:
!git clone https://github.com/clovaai/deep-text-recognition-benchmark.git

Cloning into 'deep-text-recognition-benchmark'...
remote: Enumerating objects: 499, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 499 (delta 0), reused 1 (delta 0), pack-reused 495
Receiving objects: 100% (499/499), 3.07 MiB | 18.95 MiB/s, done.
Resolving deltas: 100% (301/301), done.


In [2]:
%cd deep-text-recognition-benchmark

/content/deep-text-recognition-benchmark


In [3]:
!gdown 1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9

Downloading...
From (original): https://drive.google.com/uc?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9
From (redirected): https://drive.google.com/uc?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9&confirm=t&uuid=8eb47e9c-7e6e-4561-aa86-b4f6b2bee9b2
To: /content/deep-text-recognition-benchmark/TPS-ResNet-BiLSTM-Attn.pth
100% 199M/199M [00:01<00:00, 128MB/s]


In [5]:
!python3 demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder demo_image/ \
--saved_model TPS-ResNet-BiLSTM-Attn.pth

model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from TPS-ResNet-BiLSTM-Attn.pth
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
demo_image/demo_1.png    	available                	0.9999
demo_image/demo_2.jpg    	shakeshack               	0.9530
demo_image/demo_3.png    	london               

In [ ]:
!python3 create_lmdb_dataset.py --inputPath data/ --gtFile data/gt_train.txt --outputPath dataset/train

In [ ]:
!python3 create_lmdb_dataset.py --inputPath data/ --gtFile data/gt_validation.txt --outputPath dataset/validation

In [ ]:
!python3 train.py \
--train_data dataset/train --valid_data dataset/validation \
--select_data / --batch_ratio 1 --batch_max_length 8 --batch_max_length 100 \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn